# New Section

# Selat Sunda ANUGA 2022

## Notebook 1: Introduction to ANUGA

Here we introduce the idea of creating a `domain` which contains the mesh and quantities needed to run the simulation, and encapsulates the methods for setting up the initial conditions, the boundary conditions and the method for evolving the solution. 

These notebooks have been designed to run in the google `colaboratory` environment, which provides a jupyter notebook environment running on a virtual machine on the cloud. To use this environment you need a google account so that your notebook can be saved on google drive. 

To start interacting with the notebook follow the 
`View in Colaboratory` link above. 

## Setup Environment

If on github, first follow the link `View in Colaboratory' to start running on google's colab environment. Then ....

Run the following cell to install the dependencies and some extra code for visualising on Colaboratory.

Wait until you see the comment *(5) Ready to go* before proceeding to subsequent commands. 

The install should take less than a minute (and quicker if you have already run this earlier).

In [2]:
try:
  # On colab we can install all the packages we need via the notebook
  #
  # First download the clinic repository
  import os
  os.chdir('/content')
  !git clone https://github.com/stoiver/anuga-clinic-2018.git
  !pip install m
  # Now install environment using tool
  !/bin/bash /content/anuga-clinic-2018/anuga_tools/install_anuga_colab.sh
  !pip install mpi4py
  !pip install pymetis
  !pip install meshpy
  !pip install backports.zoneinfo
except:
  pass

# Make anuga-clinic code available
import os
if not 'workbookDir' in globals():
    workbookDir = os.getcwd()
print(workbookDir)
import sys
sys.path.append(os.path.join(workbookDir,"anuga-clinic-2018"))

# Setting Python PATH
%env PYTHONPATH="$/env/python:/content/anuga-clinic-2018"
! echo $PYTHONPATH

Cloning into 'anuga-clinic-2018'...
remote: Enumerating objects: 674, done.
remote: Total 674 (delta 0), reused 0 (delta 0), pack-reused 674
Receiving objects: 100% (674/674), 11.07 MiB | 24.65 MiB/s, done.
Resolving deltas: 100% (330/330), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement m (from versions: none)
ERROR: No matching distribution found for m
(1) Install nose via pip
(2) Install gdal via apt-get
(3) Install netcdf4 via apt-get
(4) Download anuga_core github repository
(5) Install anuga
(6) Ready to go
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 4.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp38-cp38-linux_x86_64.whl si

### Setup inline graphics and animation

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import time
import os
import sys
from osgeo import gdal
%matplotlib inline

# Allow inline jshtml animations
from matplotlib import rc
rc('animation', html='jshtml')

## Import ANUGA

Allows us access to `anuga` and inline plotting. 

In [4]:
import anuga

# Setting The Runtime Parameters

In [9]:
#------------------------------------------------------------------------------
# Runtime parameters
#------------------------------------------------------------------------------
cache = False
verbose = True

#------------------------------------------------------------------------------
# Define scenario as either slide or fixed_wave. Choose one.
#------------------------------------------------------------------------------
#scenario = 'fixed_wave'  # Wave applied at the boundary
scenario = 'slide'       # Slide wave form applied inside the domain

data_dir = '/content/sample_data/Pandeglang'
#------------------------------------------------------------------------------
# Filenames
#------------------------------------------------------------------------------
name_stem = 'SelatSunda_UTM'
meshname = name_stem + '.msh'


# Filename for locations where timeseries are to be produced
gauge_filename = 'gauges.csv'

#------------------------------------------------------------------------------
# Domain definitions
#------------------------------------------------------------------------------
# bounding polygon for study area
batas=os.path.join(data_dir,'Batas1.csv')
bounding_polygon = anuga.read_polygon(batas)

Ar = anuga.polygon_area(bounding_polygon) / 1000000.0
print ('Area of bounding polygon = %.2f km^2' % Ar)

#------------------------------------------------------------------------------
# Interior region definitions
#------------------------------------------------------------------------------
# Read interior polygons
poly_Jawa = anuga.read_polygon('/content/sample_data/Pandeglang/Jawa1.csv')
poly_island1 = anuga.read_polygon('/content/sample_data/Pandeglang/Island2.csv')
poly_island2 = anuga.read_polygon('/content/sample_data/Pandeglang/Island3.csv')
poly_island3 = anuga.read_polygon('/content/sample_data/Pandeglang/Island4.csv')
poly_island4 = anuga.read_polygon('/content/sample_data/Pandeglang/Island5.csv')
poly_island5 = anuga.read_polygon('/content/sample_data/Pandeglang/Island118.csv')
poly_island6 = anuga.read_polygon('/content/sample_data/Pandeglang/Island119.csv')
poly_island7 = anuga.read_polygon('/content/sample_data/Pandeglang/Island117.csv')
poly_shallow = anuga.read_polygon('/content/sample_data/Pandeglang/ShallowWater1.csv')

Area of bounding polygon = 4250.70 km^2


In [10]:
# Optionally plot points making up these polygons
#plot_polygons([bounding_polygon, poly_cairns, poly_island0, poly_island1,
#               poly_island2, poly_island3, poly_shallow],
#               style='boundingpoly', verbose=False)

# Define resolutions (max area per triangle) for each polygon
# Make these numbers larger to reduce the number of triangles in the model,
# and hence speed up the simulation

# bigger base_scale == less triangles
just_fitting = False
#base_scale = 25000 # 635763 # 112sec fit
#base_scale = 50000 # 321403 # 69sec fit
base_scale = 100000 # 162170 triangles # 45sec fit
#base_scale = 400000 # 42093
default_res = 100 * base_scale   # Background resolution
islands_res = base_scale
cairns_res = base_scale
shallow_res = 5 * base_scale

# Define list of interior regions with associated resolutions
interior_regions = [[poly_island1, islands_res],
                    [poly_island2, islands_res],
                    [poly_island3, islands_res],
                    [poly_island4, islands_res],
                    [poly_island5, islands_res],
                    [poly_island6, islands_res],
                    [poly_island7, islands_res],
                    [poly_shallow, shallow_res]]

#------------------------------------------------------------------------------
# Data for exporting ascii grid
#------------------------------------------------------------------------------

eastingmin = 362077.75805536134
#eastingmax = 418000
eastingmax = 636995.145998007
northingmin = 9445698.25643422
northingmax = 9170780.868491571

#------------------------------------------------------------------------------
# Data for landslide
#------------------------------------------------------------------------------
slide_origin = [547691.6153565979, 9326241.144928942]   # Assume to be on continental shelf
# slide_origin = [366242.9889088538, 9313428.814933326]   # Assume to be on continental shelf
slide_depth = 500.


#------------------------------------------------------------------------------
# Data for Tides
#------------------------------------------------------------------------------
tide = 0.0

Script for running a tsunami inundation scenario for Cairns, QLD Australia.

Source data such as elevation and boundary data is assumed to be available in directories specified above. 

The output sww file is stored in directory named after the scenario, i.e slide or fixed_wave.

The scenario is defined by a triangular mesh created from project.polygon, the elevation data and a tsunami wave generated by a submarine mass failure.

Geoscience Australia, 2004-present



In [12]:
time00 = time.time()

t_0=(time00/(3600*24)-int(time00/(3600*24)))*24
hours = t_0+7
seconds = hours * 3600
m, s = divmod(seconds, 60)
h, m = divmod(m, 60)
print ('Begin Time: %d:%02d:%.2f'%(h, m, s))

Begin Time: 7:29:14.72


# Preparation of topographic data
Convert ASC 2 DEM 2 PTS using source data and store result in source data


In [13]:
# Unzip asc from zip file
# import zipfile as zf
# if project.verbose: print ('Reading ASC from cairns.zip')
# zf.ZipFile(project.name_stem+'.zip').extract(project.name_stem+'.asc')

# Create DEM from asc data
data_asc=os.path.join(data_dir,name_stem+'.asc')
print(data_asc)
anuga.asc2dem(data_asc, use_cache=cache, verbose=verbose)

# Create pts file for onshore DEM
data_dem=os.path.join(data_dir,name_stem+'.dem')
anuga.dem2pts(os.path.join(data_dem), use_cache=cache, verbose=verbose)



CRITICAL:root:Reading METADATA from /content/sample_data/Pandeglang/SelatSunda_UTM.prj


/content/sample_data/Pandeglang/SelatSunda_UTM.asc
Reading METADATA from /content/sample_data/Pandeglang/SelatSunda_UTM.prj


CRITICAL:root:Reading DEM from /content/sample_data/Pandeglang/SelatSunda_UTM.asc


Reading DEM from /content/sample_data/Pandeglang/SelatSunda_UTM.asc


CRITICAL:root:Got 1506 lines


...

There are 2250000 values in the elevation


CRITICAL:root:There are 2250000 values in the clipped elevation


There are 2250000 values in the clipped elevation


CRITICAL:root:There are 0 NODATA_values in the clipped elevation


There are 0 NODATA_values in the clipped elevation


Create the triangular mesh and domain based on overall clipping polygon with a tagged boundary and interior regions as defined above


In [14]:
domain = anuga.create_domain_from_regions(bounding_polygon,
                                    boundary_tags={'top': [0],
                                                   'east': [1],
                                                   'ocean_south': [2],
                                                   'west': [3]},
                                    maximum_triangle_area=default_res,
                                    mesh_filename=os.path.join(data_dir,meshname),
                                    interior_regions=interior_regions,
                                    use_cache=cache,
                                    verbose=verbose)

# Print some stats about mesh and domain
print ('Number of triangles = ', len(domain))
print ('The extent is ', domain.get_extent())
print (domain.statistics())
                                    
time01 = time.time()
t_0=(time01/(3600*24)-int(time01/(3600*24)))*24
h0=h;m0=m;s0=s
hours = t_0+7
seconds = hours * 3600
m, s = divmod(seconds, 60)
h, m = divmod(m, 60)
print ('Second Time: %d:%02d:%02d'%(h, m, s))
print ('Consumed Time: %d:%02d:%.2f'%(h-h0, m-m0, s-s0))

CRITICAL:root:Resource usage: memory=1266.1MB resident=337.1MB stacksize=0.1MB


Resource usage: memory=1266.1MB resident=337.1MB stacksize=0.1MB


/usr/local/lib/python3.8/dist-packages/anuga/pmesh/mesh_interface.py:181: UserWarning: Segment 4 was not assigned a boundary_tag.Default tag "exterior" will be assigned to missing segment
  warnings.warn(msg, UserWarning)
CRITICAL:root:WARNING: Segment 4 was not assigned a boundary_tag.Default tag "exterior" will be assigned to missing segment


CRITICAL:root:Generating mesh to file '/content/sample_data/Pandeglang/SelatSunda_UTM.msh'


Streaming output truncated to the last 5000 lines.
  4163: 8955,3080
  4164: 8956,2906
  4165: 7387,4142
  4166: 8957,4058
  4167: 8985,11
  4168: 6614,209
  4169: 8988,206
  4170: 8989,2543
  4171: 8990,9417
  4172: 6619,2572
  4173: 8995,1556
  4174: 8997,2361
  4175: 8998,2358
  4176: 6653,4714
  4177: 9004,1353
  4178: 6745,10230
  4179: 7007,1672
  4180: 7091,2159
  4181: 7636,9044
  4182: 9216,9472
  4183: 9297,829
  4184: 9361,9509
  4185: 8884,1439
  4186: 9413,6614
  4187: 9416,8989
  4188: 9417,2541
  4189: 8993,6619
  4190: 9422,8997
  4191: 9423,8998
  4192: 9472,1237
  4193: 9482,9004
  4194: 8803,4802
  4195: 9509,1308
  4196: 9520,8994
  4197: 8991,9541
  4198: 9303,8803
  4199: 9541,2574
  4200: 9542,9303
  4201: 9584,2159
  4202: 7043,269
  4203: 7044,272
  4204: 9617,338
  4205: 9616,10104
  4206: 9618,240
  4207: 10107,158
  4208: 9620,169
  4209: 9621,2592
  4210: 9622,10111
  4211: 9623,2419
  4212: 7057,2422
  4213: 8393,1216
  4214: 9627,1234
  4215: 7076,1326
  

...

Domain: Set work arrays


CRITICAL:root:Domain: Initialising quantity values


Domain: Initialising quantity values


CRITICAL:root:Domain: Done


Domain: Done
##########################################################################
#
# Using discontinuous elevation solver DE0
#
# First order timestepping
#
# Make sure you use centroid values when reporting on important output quantities
#
##########################################################################
Number of triangles =  38837
The extent is  (0.0, 66684.34489779244, 0.0, 63902.2492219042)
------------------------------------------------
Mesh statistics:
  Number of triangles = 38837
  Extent [m]:
    x in [0.00000e+00, 6.66843e+04]
    y in [0.00000e+00, 6.39022e+04]
  Areas [m^2]:
    A in [7.15256e-06, 9.95035e+06]
    number of distinct areas: 38837
    Histogram:
      [7.15256e-06, 9.95035e+05[: 38233
      [9.95035e+05, 1.99007e+06[: 169
      [1.99007e+06, 2.98510e+06[: 64
      [2.98510e+06, 3.98014e+06[: 49
      [3.98014e+06, 4.97517e+06[: 71
      [4.97517e+06, 5.97021e+06[: 78
      [5.97021e+06, 6.96524e+06[: 73
      [6.96524e+06, 7.96028e+06[: 56
 

In [15]:
#------------------------------------------------------------------------------
# Setup parameters of computational domain
#------------------------------------------------------------------------------
domain.set_name('SelatSunda_' + scenario)     # Name of sww file
domain.set_datadir(data_dir)                  # Store sww output here
domain.set_minimum_storable_height(0.01)      # Store only depth > 1cm
domain.set_flow_algorithm('DE0')


                                    
#------------------------------------------------------------------------------
# Setup initial conditions
#------------------------------------------------------------------------------
tide = tide
domain.set_quantity('stage', tide)
domain.set_quantity('friction', 0.0)


domain.set_quantity('elevation', 
                    filename=os.path.join(data_dir,name_stem + '.pts'),
                    use_cache=cache,
                    verbose=verbose,
                    alpha=0.1)


time02 = time.time()
t_0=(time02/(3600*24)-int(time02/(3600*24)))*24
h10=h;m10=m;s10=s
hours = t_0+7
seconds = hours * 3600
m, s = divmod(seconds, 60)
h, m = divmod(m, 60)
print ('Second Time: %d:%02d:%02d'%(h, m, s))
print ('Consumed Time: %d:%02d:%.2f'%(h-h10, m-m10, s-s10))
print ('That took %.2f seconds to fit data' %(time01-time02))

if just_fitting:
    import sys
    sys.exit()

#------------------------------------------------------------------------------
# Setup information for slide scenario (to be applied 1 min into simulation
#------------------------------------------------------------------------------
if scenario == 'slide':
    # Function for submarine slide
    tsunami_source = anuga.slide_tsunami(length=35000.0,
                                   depth=slide_depth,
                                   slope=6.0,
                                   thickness=500.0, 
                                   x0=slide_origin[0], 
                                   y0=slide_origin[1], 
                                   alpha=0.0, 
                                   domain=domain,
                                   verbose=verbose)

#------------------------------------------------------------------------------
# Setup boundary conditions
#------------------------------------------------------------------------------
print ('Available boundary tags', domain.get_boundary_tags())

Bd = anuga.Dirichlet_boundary([tide, 0, 0]) # Mean water level
Bs = anuga.Transmissive_stage_zero_momentum_boundary(domain) # Neutral boundary

if scenario == 'fixed_wave':
    # 10m wave starting after 60 seconds and lasting 1 hour.
    Bw = anuga.Transmissive_n_momentum_zero_t_momentum_set_stage_boundary(
                        domain=domain, 
                        function=lambda t: [(60<t<3660)*10, 0, 0])

    domain.set_boundary({'top': Bd,
                         'east': Bs,
                         'ocean_south': Bw,
                         'west': Bs})

if scenario == 'slide':
    # Boundary conditions for slide scenario
    domain.set_boundary({'top': Bd,
                         'east': Bd,
                         'ocean_south': Bd,
                         'west': Bd})

time102 = time.time()
t_0=(time102/(3600*24)-int(time102/(3600*24)))*24

hours = t_0+7
seconds = hours * 3600
m, s = divmod(seconds, 60)
h, m = divmod(m, 60)
print ('Second Time: %d:%02d:%02d'%(h, m, s))
print ('Consumed Time: %d:%02d:%.2f'%(h-h0, m-m0, s-s0))

CRITICAL:root:FitInterpolate: Building quad tree


##########################################################################
#
# Using discontinuous elevation solver DE0
#
# First order timestepping
#
# Make sure you use centroid values when reporting on important output quantities
#
##########################################################################
FitInterpolate: Building quad tree


CRITICAL:root:Building smoothing matrix


Building smoothing matrix


CRITICAL:root:Geospatial_data: Created from file: /content/sample_data/Pandeglang/SelatSunda_UTM.pts


...

	3D amplitude: 179.07967943022047


CRITICAL:root:	scale for eta(x,y): 206.86020289657608


	scale for eta(x,y): 206.86020289657608


CRITICAL:root:WARNING: d/b out of range (0.06 - 1.5) 0.014285714285714285


Available boundary tags ['top', 'east', 'ocean_south', 'west']
Second Time: 7:30:34
Consumed Time: 0:01:19.49


In [ ]:
#------------------------------------------------------------------------------
# Evolve system through time
#------------------------------------------------------------------------------

time03 = time.time()
t_0=(time03/(3600*24)-int(time03/(3600*24)))*24
hours = t_0+7
seconds = hours * 3600
m, s = divmod(seconds, 60)
h, m = divmod(m, 60)
print ('Begin Time: %d:%02d:%.2f'%(h, m, s))
from numpy import allclose

if scenario == 'slide':
    # Initial run without any event
    for t in domain.evolve(yieldstep=10, finaltime=60): 
        print (domain.timestepping_statistics())
        print( domain.boundary_statistics(tags='ocean_south')   )     
        
    # Add slide to water surface
    if allclose(t, 60):
        domain.add_quantity('stage', tsunami_source)    

    # Continue propagating wave
    for t in domain.evolve(yieldstep=10, finaltime=5000, 
                           skip_initial_step=True):
        print( domain.timestepping_statistics())
        print( domain.boundary_statistics(tags='ocean_south')    )

if scenario == 'fixed_wave':
    # Save every two mins leading up to wave approaching land
    for t in domain.evolve(yieldstep=2*60, finaltime=5000): 
        print (domain.timestepping_statistics())
        print (domain.boundary_statistics(tags='ocean_south') )   

    # Save every 30 secs as wave starts inundating ashore
    for t in domain.evolve(yieldstep=60*0.5, finaltime=10000, 
                           skip_initial_step=True):
        print (domain.timestepping_statistics())
        print (domain.boundary_statistics(tags='ocean_south'))
time04=time.time()
h1=h;m1=m;s1=s
t_1=(time04/(3600*24)-int(time04/(3600*24)))*24
hours = t_1+7
seconds = hours * 3600
m, s = divmod(seconds, 60)
h, m = divmod(m, 60)
print ('End Time: %d:%02d:%02d'%(h, m, s))            
print ('That took %.2f seconds' %(time00-time04))
print ('Consumed Time this section: %d:%02d:%02d'%(h-h1, m-m1, s-s1))
print ('Consumed Time: %d:%02d:%.2f'%(h-h0, m-m0, s-s0))

Begin Time: 7:30:34.25
Cumulative mass protection: 117893961588.56389 m^3
Time = 0.0000 (sec), steps=0 (3s)
Boundary values at time 0.0000:
    ocean_south:
        stage     in [  0.00000000,   0.00000000]
        xmomentum in [  0.00000000,   0.00000000]
        ymomentum in [  0.00000000,   0.00000000]

Time = 10.0000 (sec), delta t in [0.00004246, 0.00004246] (s), steps=235521 (7541s)
Boundary values at time 10.0000:
    ocean_south:
        stage     in [  0.00000000,   0.00000000]
        xmomentum in [  0.00000000,   0.00000000]
        ymomentum in [  0.00000000,   0.00000000]



## Plot Mesh

Let's look at the mesh. We will use some code derived form the `clawpack` project to simply plot and animate the output from our simulations. This is available via the `animate` module loaded from `anuga`.

The `Domain_plotter` class provides a plotting wrapper around our standard `anuga` `domain`, providing simple access to the centroid values of our evolution quantities, `stage`, `depth`, `elev`, `xmon` and `ymon` and the triangulation `triang`.


In [ ]:
dplotter1 = anuga.Domain_plotter(domain)  
plt.triplot(dplotter1.triang, linewidth = 0.4);

### View Elevation

Let's use the `matplotlib` function `tripcolor` to plot the `elevation` quantitiy.  We access the `domain` mesh and elevation quantitiy via the `dplotter` interface.  

In [ ]:
plt.tripcolor(dplotter1.triang, 
              facecolors = dplotter1.elev, 
              edgecolors='k', 
              cmap='Greys_r')
plt.colorbar();

## Run the Evolution

We evolve using a `for` statement, which evolves the quantities using the shallow water wave solver. The calculation `yields` every `yieldstep` seconds, for a given `duration`.

In [ ]:
for t in domain.evolve(yieldstep=2, duration=40):
  
    #dplotter.plot_depth_frame()
    dplotter1.save_depth_frame(vmin=0.0,vmax=1.0)
    
    domain.print_timestepping_statistics()

    
# Read in the png files stored during the evolve loop
dplotter1.make_depth_animation() 

### Optional Installation of ANUGA Viewer

On  Windows you might like to download and extract a precompiled version ``anuga_viewer.zip`` from sourceforge https://sourceforge.net/projects/anuga/files/anuga_viewer_windows/

Once extracted, go to the data directory and associate the sww file with the viewer executable in the bin directory. 

Then sww files should always open with the anuga viewer.

### Download Domain 

If you have the ``anuga_viewer`` installed on your local machine you might want to download the output file ``domain1.sww`` to your local disk and view it using the ``anuga_viewer``

I had problems on using the Chrome browser and had to follow the instructions from this site to allow downloads, https://stackoverflow.com/questions/53581023/google-colab-file-download-failed-to-fetch-error

In [ ]:
files.download('domain.sww')